In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:
dataset_path = "/kaggle/input/minor-dataset/Data_Minor"

classes = ["HR", "DR", "RVO"]
num_classes = len(classes)

img_size = (224, 224)
input_shape = (224, 224, 3)

batch_size = 8
epochs = 25


In [ ]:
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(2.0, (8,8))
    l = clahe.apply(l)
    return cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2RGB)

def preprocess_image(image, use_preprocessing=True):
    if use_preprocessing:
        image = apply_clahe(image)
    return image.astype(np.float32) / 255.0


In [ ]:
def load_images(use_preprocessing=True):
    images, labels = [], []
    for label, cls in enumerate(classes):
        folder = os.path.join(dataset_path, cls)
        for f in os.listdir(folder):
            img = cv2.imread(os.path.join(folder, f))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, img_size)
            img = preprocess_image(img, use_preprocessing)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)


In [ ]:
def get_split(images, labels):
    return train_test_split(
        images, labels,
        test_size=0.2,
        stratify=labels,
        random_state=42
    )


In [ ]:
def build_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, 3, activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [ ]:
def transformer_block(x, num_heads=4, ff_dim=128):
    attn = layers.MultiHeadAttention(num_heads, key_dim=x.shape[-1])(x, x)
    x = layers.Add()([x, attn])
    x = layers.LayerNormalization()(x)

    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(x.shape[-1])(ff)
    x = layers.Add()([x, ff])
    return layers.LayerNormalization()(x)


In [ ]:
def build_transformer(input_shape, num_classes, patch_size=16, embed_dim=64):
    inputs = layers.Input(shape=input_shape)

    # Patch extraction
    patches = layers.Conv2D(
        embed_dim,
        kernel_size=patch_size,
        strides=patch_size,
        padding="valid"
    )(inputs)

    # Flatten patches
    x = layers.Reshape((-1, embed_dim))(patches)

    # Transformer encoder
    for _ in range(2):
        x = transformer_block(x, num_heads=4, ff_dim=128)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)


In [ ]:
def build_hybrid(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # CNN feature extractor
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)

    # Flatten spatial → tokens
    x = layers.Reshape((-1, 64))(x)

    # Transformer encoder
    for _ in range(2):
        x = transformer_block(x, num_heads=4, ff_dim=128)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)


In [ ]:
def train_and_eval(model, X_train, y_train, X_test, y_test):
    model.compile(
        optimizer=Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, to_categorical(y_train, num_classes),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    preds = np.argmax(model.predict(X_test), axis=1)
    return accuracy_score(y_test, preds)


In [ ]:
results = []

for use_preprocessing in [False, True]:
    images, labels = load_images(use_preprocessing)
    X_train, X_test, y_train, y_test = get_split(images, labels)

    for name, builder in [
        ("CNN Only", build_cnn),
        ("Transformer Only", build_transformer),
        ("Hybrid CNN+Transformer", build_hybrid)
    ]:
        model = builder(input_shape, num_classes)
        acc = train_and_eval(model, X_train, y_train, X_test, y_test)

        results.append([
            name,
            "Yes" if use_preprocessing else "No",
            f"{acc*100:.2f}%"
        ])

        print(f"{name} | Preprocessing: {use_preprocessing} | Acc: {acc:.4f}")


In [ ]:
import pandas as pd

df = pd.DataFrame(
    results,
    columns=["Model", "Preprocessing", "Test Accuracy"]
)

print("\nABLATION STUDY RESULTS")
print(df.to_markdown(index=False))
